<a href="https://colab.research.google.com/github/JackChin3/AIT_DeepLearning_4/blob/main/JackChin_DeepLearning_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the UCI Covertype Dataset

!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
covertype = fetch_ucirepo(id=31)

# data (as pandas dataframes)
x = covertype.data.features
y = covertype.data.targets

# metadata
print(covertype.metadata)

# variable information
print(covertype.variables)


{'uci_id': 31, 'name': 'Covertype', 'repository_url': 'https://archive.ics.uci.edu/dataset/31/covertype', 'data_url': 'https://archive.ics.uci.edu/static/public/31/data.csv', 'abstract': 'Classification of pixels into 7 forest cover types based on attributes such as elevation, aspect, slope, hillshade, soil-type, and more.', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 581012, 'num_features': 54, 'feature_types': ['Categorical', 'Integer'], 'demographics': [], 'target_col': ['Cover_Type'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1998, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C50K5N', 'creators': ['Jock Blackard'], 'intro_paper': None, 'additional_info': {'summary': 'Predicting forest cover type from cartographic variables only (no remotely sensed data).  The actual forest cover type for a given observation (30 x 30 meter cell) was determined from

In [3]:
# Checking the dimensions of the data, and reconfirming no missing values
print(x.shape)
print(y.shape)
print(x.isnull().sum())


(581012, 54)
(581012, 1)
Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0
Soil_Type12                           0
Soil_Type13                           0
Soil_Type14    

In [4]:
# Splitting into 80% training, 20% validation set

train_ratio  = 0.8
train_length = x.shape[0]
train_split  = int(train_ratio * train_length)
X_valid = x.iloc[train_split:, :]
Y_valid = y.iloc[train_split:, :]
X_train = x.iloc[:train_split, :]
Y_train = y.iloc[:train_split, :]

In [5]:
# According to the dataset metadata, these are the numerical features that need to be normalized
numerical_features = [
    'Elevation', 'Aspect', 'Slope',
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]

In [6]:
import pandas as pd

# Normalizing only the numerical features, based on the SD and mean from the training set

X_train_num = X_train[numerical_features]
X_valid_num = X_valid[numerical_features]

X_train_cat = X_train.drop(columns=numerical_features)
X_valid_cat = X_valid.drop(columns=numerical_features)

train_mean = X_train_num.mean()
train_std = X_train_num.std()

X_train_num_norm = (X_train_num - train_mean) / train_std
X_valid_num_norm = (X_valid_num - train_mean) / train_std

X_train_processed = pd.concat([X_train_num_norm, X_train_cat], axis=1)
X_valid_processed = pd.concat([X_valid_num_norm, X_valid_cat], axis=1)

print(X_train_processed.head())

   Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0  -1.222667 -0.906362 -1.424446                         -0.022410   
1  -1.244227 -0.861537 -1.557412                         -0.244466   
2  -0.475270 -0.117436 -0.626646                          0.025863   
3  -0.543542  0.026005  0.570052                         -0.099647   
4  -1.226261 -0.960152 -1.557412                         -0.529276   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                       -0.807019                        -1.237620   
1                       -0.914431                        -1.312195   
2                        0.356617                         0.421673   
3                        1.305427                         0.365742   
4                       -0.824921                        -1.311574   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0       0.306237        0.453588       0.164533   
1       0.268683        0.606799       0.244017   
2    

In [7]:
# Encoding the categorical variables (and changed to 0-indexed)

from tensorflow.keras.utils import to_categorical

Y_train_flat = (Y_train.values - 1).reshape(-1)
Y_valid_flat = (Y_valid.values - 1).reshape(-1)

Y_train_encoded = to_categorical(Y_train_flat)
Y_valid_encoded = to_categorical(Y_valid_flat)

print("Y_train_encoded shape:", Y_train_encoded.shape)
print("Y_valid_encoded shape:", Y_valid_encoded.shape)

Y_train_encoded shape: (464809, 7)
Y_valid_encoded shape: (116203, 7)


In [8]:
# Model Architecture:

from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.models import Model

In [9]:
# Input
InputLayer = Input(shape=(X_train_processed.shape[1],))

# Residual Block
DenseLayer1 = Dense(64, activation = 'relu')(InputLayer)
DenseLayer2 = Dense(64, activation = 'relu')(DenseLayer1)
LinearProjection = Dense(64, activation = None)(InputLayer)
ResidualConnection = Add()([DenseLayer2, LinearProjection])

# Intermediate Layer
IntermediateLayer1 = Dense(64, activation = 'relu')(ResidualConnection)
IntermediateLayer2 = Dense(64, activation = 'relu')(IntermediateLayer1)

# Skip Connection
SkipConnection = Dense(64, activation = 'relu')(ResidualConnection)
CombinedConnection = Add()([IntermediateLayer2, SkipConnection])

# Final Layers
FinalDense1 = Dense(64, activation = 'relu')(CombinedConnection)
FinalDense2 = Dense(64, activation = 'relu')(FinalDense1)

# Output Layer
OutputLayer = Dense(7, activation='softmax')(FinalDense2)

FinalModel = Model(inputs=InputLayer, outputs=OutputLayer)


In [10]:
# Saving the model

FinalModel.save('covertype_model.h5')

In [11]:
# Training

X_batch = X_train_processed[:128]
Y_batch = Y_train_encoded[:128]

FinalModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
history = FinalModel.fit(
    X_batch, Y_batch,
    epochs=200,
    batch_size=128,
    verbose=1
)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 2.3166
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.0000e+00 - loss: 2.0949
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0938 - loss: 1.9334
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4375 - loss: 1.8104
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5781 - loss: 1.7107
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.6094 - loss: 1.6278
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6094 - loss: 1.5508
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6250 - loss: 1.4746
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6406 - loss: 1.3969
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6484 - loss: 1.3172
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6484 - loss: 1.2368
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.656

In [13]:
# Validation Check
val_loss, val_accuracy = (FinalModel.evaluate(X_valid_processed, Y_valid_encoded))

3632/3632 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.4218 - loss: 4.0636


In [14]:
print(val_loss)
print(val_accuracy)

4.8700361251831055
0.398784875869751


In [15]:
# Conclusion:
print("Number of Parameters: " + str(FinalModel.count_params()))
print("Final Training Loss: " + str(history.history['loss'][-1]))
print("Final Validation Loss: " + str(val_loss))


Number of Parameters: 32455
Final Training Loss: 0.0007368979859165847
Final Validation Loss: 4.8700361251831055
